In [54]:
import os
import json
import facebook
import csv
import datetime
token = 'EAAitbsxNgPUBALm5yR0vhsnZBNHyH122B4KVb2dxLN0ZCOHIYkPdBdTfXNvXPZAD1XCXv1lZAscnBBynxhfTqRZAohSL2cdQqCHoW5gF08NqaQMJRN97AtvW2sv6uKVkpPoNZBVP9ocFjZATtTCmkmPLZAspeZBn79tIJQq1wh1bPjgZDZD'
def profile():
    graph = facebook.GraphAPI(token)
    profile = graph.get_object('me', fields='name,location{location}')
    print(json.dumps(profile, indent=4))
    
def get_friends():
    graph = facebook.GraphAPI(token)
    user = graph.get_object("me")
    friends = graph.get_connections(user["id"], "friends")
    print(json.dumps(friends, indent=4))


def get_my_posts():
    fields = [
        'message',
        'created_time',
        'description',
        'caption',
        'link',
        'likes.limit(1).summary(true)',
        'comments.limit(1).summary(true)',
        'shares',
        'place',
        'status_type'
    ]
    all_fields = ','.join(fields)
    graph = facebook.GraphAPI(token)
    posts = graph.get_connections('me', 'feed', fields=all_fields)
#     print(posts)
    return posts
def get_page(page_name):
    fields = ['id','name','about','likes','website','link']
    graph = facebook.GraphAPI(token)
    fields = [
        'message',
        'created_time',
        'description',
        'caption',
        'link',
        'likes.limit(1).summary(true)',
        'comments.limit(1).summary(true)',
        'shares',
        'place',
        'status_type'
    ]
    all_fields = ','.join(fields)
    posts = graph.get_connections(page_name, 'posts', fields=all_fields)
    return posts
def FeedStatus(status):    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
#     link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['status_type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

def scrapeFeedStatus():     
    has_next_page = True
    num_processed = 0   # keep a count on how many we've processed
    scrape_starttime = datetime.datetime.now()

    print("Scraping Facebook : %s\n" % (scrape_starttime))
    statuses = get_my_posts()
#     page_name = input('Enter the page naame you want to search:' )
#     statuses = get_page(page_name)
    
    while has_next_page:
        for status in statuses['data']:
#             print(status)
            output = FeedStatus(status)
            print(output)

            # output progress occasionally to make sure code is not stalling
            num_processed += 1
            if num_processed % 1000 == 0:
                print("%s Statuses Processed: %s" % (num_processed, datetime.datetime.now()))

        # if there is no next page, we're done.
        if 'paging' in statuses.keys():
            statuses = requests.get(statuses['paging']['next']).json()
        else:
            has_next_page = False


    print ("\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime))

if __name__ == '__main__':
#     token = os.environ.get('FACEBOOK_TEMP_TOKEN')
#     get_friends()
#     profile()
#     get_my_posts()
    scrapeFeedStatus()


Scraping Facebook : 2019-06-17 17:03:36.330656

('2483284935036473_2502242226474077', '', 'mobile_status_update', '', '2019-06-17 00:32:40', 2, 0, 0)
('2483284935036473_2500324826665817', b'Awesome one.\n#GoogleDoodle', 'shared_story', 'https://g.co/doodle/wy3g2x', '2019-06-16 00:26:27', 2, 0, 0)
('2483284935036473_2500313306666969', b'Thank you @Andela @googleafrica @pluralsight for this great opportunity. I embrace it and let the happy coding continue.\n\n#ALCwithGoogle #growwithgoogle #pluralsight', 'added_photos', 'https://www.facebook.com/photo.php?fbid=2500312520000381&set=a.607857889245863&type=3', '2019-06-16 00:15:57', 6, 1, 0)
('2483284935036473_2492942814070685', '', 'mobile_status_update', '', '2019-06-12 03:47:41', 4, 1, 0)
('2483284935036473_2491738880857745', '', 'shared_story', 'https://www.gofundme.com/help-me-have-a-good-wending?pc=fb_co_shareflow_m&rcid=r01-156020165815-9247d7028a5140c2&utm_source=facebook&utm_medium=social&utm_campaign=fb_co_shareflow_m', '2019-06-1

('2483284935036473_1937135346318104', b'You are the temple of God. (1 Corinthians 3:16) And your temple should be a \xe2\x80\x9chouse of prayer\xe2\x80\x9d or it becomes a \xe2\x80\x9cden of thieves\xe2\x80\x9d (Mark 11:17). These thieves or powers of darkness, will steal and rob Your strength, health, youthfulness, wisdom, hopes and dreams (1 Corinthians 6:19)', 'mobile_status_update', '', '2018-06-22 05:15:56', 1, 0, 0)
('2483284935036473_1935596053138700', '', 'mobile_status_update', 'https://www.facebook.com/Lazzomint/videos/869180986599715/', '2018-06-21 06:27:46', 0, 0, 0)
('2483284935036473_1935539006477738', b'Always establish the truth of God\xe2\x80\x99s Word and Christ\xe2\x80\x99s finished work over your problem.', 'mobile_status_update', '', '2018-06-21 05:36:46', 4, 0, 0)
('2483284935036473_1933734916658147', b'SEE THE MIRACULOUS WHEN YOU GIVE THANKS.\n\nAnd Jesus lifted up His eyes and said, \xe2\x80\x9cFather, I thank You that You have heard Me. \n\nJohn 11:41.\n\nYour 

('2483284935036473_1466991753332468', b'Liabilities: costs you Money\nAssets: pays you money.\nBillionaires words definition.', 'mobile_status_update', '', '2017-03-29 09:36:27', 1, 0, 0)
('2483284935036473_1459513397413637', '', 'added_photos', 'https://www.facebook.com/photo.php?fbid=1459513347413642&set=a.151385458226444&type=3', '2017-03-23 07:02:16', 43, 16, 0)
('2483284935036473_1452380041460306', b'Thank you God for Everything. You always there for me.', 'mobile_status_update', '', '2017-03-16 22:23:26', 6, 0, 0)
('2483284935036473_1443057352392575', b'Thanks you all for you birthday wishes. You have been of much help in my life.', 'mobile_status_update', '', '2017-03-08 07:54:08', 3, 0, 0)
('2483284935036473_1431420290222948', b'Up and up. Mpaka kileleni.', 'mobile_status_update', '', '2017-02-25 03:33:36', 2, 0, 0)
('2483284935036473_1422941841070793', b'Staying down after a fall is a matter choice', 'mobile_status_update', '', '2017-02-17 03:44:34', 3, 0, 0)
('248328493503647

('2483284935036473_604340592930926', b'one smile can start a friendship, one word can stop a fight, one look can save a relationship, one person can change your life.', 'mobile_status_update', '', '2013-07-02 02:07:22', 2, 0, 0)
('2483284935036473_603164666381852', b'It does not matter the degrees you have, its just a section in a circumfrence but God is the whole circumfrence.', 'mobile_status_update', '', '2013-06-29 13:57:10', 2, 2, 0)
('2483284935036473_583879864976999', b'Holy spirit is a bulldozer and not a sympathizer. W0W', 'mobile_status_update', '', '2013-05-18 09:51:44', 2, 0, 0)
('2483284935036473_579726002059052', b'Honesty pays', 'mobile_status_update', '', '2013-05-08 06:55:14', 0, 0, 0)
('2483284935036473_553976967967289', b'dhanx to all 4 HBD wishes>:)', 'mobile_status_update', '', '2013-03-08 10:43:40', 1, 1, 0)
('2483284935036473_537636669601319', '', 'published_story', 'http://www.linkedin.com/pub/kennedy-mbogo/64/b61/470?trk=fbr', '2013-02-01 09:29:54', 0, 0, 0)
('

('2483284935036473_121917744530073', b'poa poa mie aka nguch nimeingia wid da same same character as principled as i waz si mtalk mswitch kusilence hivyo.', 'mobile_status_update', '', '2010-09-30 10:55:58', 0, 0, 0)
('2483284935036473_164049256943494', b'God created everyone with a purpose!!!!!! He says. i was created with a purpose. you too was created with a purpose get to know that.', 'mobile_status_update', '', '2010-09-29 01:23:36', 0, 0, 0)
('2483284935036473_153004531400870', b'na miss kujoin chuo si time imove haraka', 'mobile_status_update', '', '2010-09-27 09:27:07', 0, 0, 0)
('2483284935036473_153607588005906', b'stone uko kwa studio ama na ina endelea kukua..............................', 'mobile_status_update', '', '2010-09-27 09:25:21', 0, 0, 0)
('2483284935036473_151385478226442', '', 'added_photos', 'https://www.facebook.com/photo.php?fbid=151385461559777&set=a.151385458226444&type=3', '2010-09-17 08:17:17', 0, 0, 0)

Done!
179 Statuses Processed in 0:00:09.843766
